In [1]:
import quandl
import pandas as pd
from matplotlib import pyplot as plt
import requests
import statsmodels.tsa.stattools as ts 
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import scipy.stats as st
from numpy_ext import rolling_apply # multipler parameters
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
symbols = pd.read_csv("crypto_ohlc/symbols.csv").symbol.tolist()
df = ((pd.concat([pd.read_csv(f"crypto_ohlc/{symbol}.csv", index_col='date', parse_dates=True)
        ['close'].rename(symbol)for symbol in symbols],axis=1,sort=True)))
df = df.loc[:,~df.columns.duplicated()]
df.head()

In [ ]:
norm_prices = df.divide(df.iloc[0])

plt.figure(figsize = (15, 10))
plt.plot(norm_prices)
plt.xlabel('days')
plt.title('Performance of cryptocurrencies')
plt.legend(symbols)
plt.show()

result_list = []
for a1 in df.columns:
    for a2 in df.columns:
        if a1 != a2:
            test_result = ts.coint(df[a1], df[a2])
            if test_result[1] < 0.05:
                print(a1 + ' and ' + a2 + ': p-value = ' + str(test_result[1]))
                result_list.append((a1, a2, test_result[1]))

In [ ]:
import statsmodels.formula.api as sm
import statsmodels.tsa.stattools as ts

def z_score(yport):
    zscore = (yport[-1]  - yport.mean()) / yport.std()
    
    return zscore

def upToOut(ANewPrice, BNewPrice, aPos, bPos):
    """
    statusList = (1,0)
    long A short B
    """
    balanceDict['A']['available'] += balanceDict['A']['position'] * (ANewPrice - AOldOrderPrice + AOldOrderPrice) 
    balanceDict['A']['position'] -= balanceDict['A']['position']
    balanceDict['B']['available'] += abs(balanceDict['B']['position']) * (BOldOrderPrice - BNewPrice + BOldOrderPrice)
    balanceDict['B']['position'] -= balanceDict['B']['position']  
    
def upToDown(ANewPrice, BNewPrice, aPos, bPos):
    """
    statusList = (1,-1)
    """
    balanceDict['A']['available'] += balanceDict['A']['position'] * (ANewPrice - AOldOrderPrice + AOldOrderPrice) 
    balanceDict['A']['position'] -= balanceDict['A']['position']
    balanceDict['B']['available'] += abs(balanceDict['B']['position']) * (BOldOrderPrice - BNewPrice + BOldOrderPrice) 
    balanceDict['B']['position'] -= balanceDict['B']['position']   

    balanceDict['A']['available'] -=  ANewPrice * abs(aPos)
    balanceDict['A']['position'] += aPos
    balanceDict['B']['available'] -= BNewPrice * abs(bPos)
    balanceDict['B']['position'] += -(bPos * aPos)
    
def outToUp(ANewPrice, BNewPrice, aPos, bPos):
    """
    statusList = (0,1)
    """
    
    balanceDict['A']['available'] -= ANewPrice * abs(aPos)
    balanceDict['A']['position'] += aPos
    balanceDict['B']['available'] -= BNewPrice * abs(bPos)
    balanceDict['B']['position'] += -(bPos * aPos)
    
def outToDown(ANewPrice, BNewPrice, aPos, bPos):
    """
    statusList = (0,-1)
    """
    balanceDict['A']['available'] -=  ANewPrice * abs(aPos)
    balanceDict['A']['position'] += aPos
    balanceDict['B']['available'] -= BNewPrice * abs(bPos)
    balanceDict['B']['position'] += -(bPos * aPos)
    
def downToOut(ANewPrice, BNewPrice, aPos, bPos):
    """
    statusList = (-1,0)
    """
    balanceDict['A']['available'] += abs(balanceDict['A']['position']) * (AOldOrderPrice - ANewPrice + AOldOrderPrice)
    balanceDict['A']['position'] -= balanceDict['A']['position']
    balanceDict['B']['available'] += balanceDict['B']['position'] * (BNewPrice - BOldOrderPrice + BOldOrderPrice)
    balanceDict['B']['position'] -= balanceDict['B']['position']  
    
def downToUp(ANewPrice, BNewPrice, aPos, bPos):
    """
    statusList = (-1,1)
    """
    balanceDict['A']['available'] += abs(balanceDict['A']['position']) * (AOldOrderPrice - ANewPrice + AOldOrderPrice)
    balanceDict['A']['position'] -= balanceDict['A']['position']
    balanceDict['B']['available'] += balanceDict['B']['position'] * (BNewPrice - BOldOrderPrice + BOldOrderPrice)
    balanceDict['B']['position'] -= balanceDict['B']['position']   
    
    balanceDict['A']['available'] -= ANewPrice * abs(aPos)
    balanceDict['A']['position'] += aPos
    balanceDict['B']['available'] -= BNewPrice * abs(bPos)
    balanceDict['B']['position'] += -(bPos * aPos)


strategy = {(1,0): upToOut, 
            (1,-1):upToDown,
            (0,1): outToUp,
            (0,-1):outToDown,
            (-1,0):downToOut,
            (-1,1):downToUp}    

In [ ]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)
# import statsmodels.formula.api as sm
# import statsmodels.tsa.stattools as ts

for i in result_list:
    A_symbol, B_symbol = i[0], i[1]

    init = 100000
    pastStatus = 0
    AOldOrderPrice = None
    BOldOrderPrice = None
    aEntryPriceList, bEntryPriceList = [], []
    lookback = 20

    balanceDict = {'A':{'available': 100000, 'position':0}, 'B':{'available': 100000, 'position':0}}
    entryZscore = 1 
    exitZscore = 0
    miniSpread = 0.01
    df1 = df[[A_symbol, B_symbol]]

    hedgeRatio=np.full(df1.shape[0], 0.0)

    df1['ratio'] = df1[A_symbol]/df1[B_symbol]
    df1['zscore'] = rolling_apply(z_score, 20, df1['ratio'])

    history = pd.DataFrame()
    statusList = []
    pl = []
    date, A_available, A_position, B_available, B_position, init, Balance , A_price, B_price= [], [], [], [], [], [], [], [], []
    for index, row in df1.iterrows():
        currStatus = 0 if (pastStatus == 1 and row['zscore'] > -entryZscore) or (pastStatus == -1 and row['zscore'] < exitZscore) else 1 if row['zscore'] < -entryZscore else -1 if (pastStatus == -1 and row['zscore'] > exitZscore or row['zscore'] > entryZscore) else 0
        con = (pastStatus, currStatus)
        if pastStatus != currStatus:
            strategy[con](row[A_symbol], row[B_symbol], currStatus, row['ratio'], )
            AOldOrderPrice, BOldOrderPrice = row[A_symbol], row[B_symbol]
            aEntryPriceList.append(AOldOrderPrice)
            bEntryPriceList.append(BOldOrderPrice)
        else:
            aEntryPriceList.append(0)
            bEntryPriceList.append(0)
        pastStatus = currStatus
        statusList.append(currStatus)

        date.append(index)
        A_price.append(row[A_symbol])
        B_price.append(row[B_symbol])
        A_available.append(balanceDict['A']['available'])
        A_position.append(balanceDict['A']['position'])
        B_available.append(balanceDict['B']['available'])
        B_position.append(balanceDict['B']['position'])
        Balance.append(balanceDict['A']['available'] + balanceDict['B']['available'])
    #     print('date: ', index, 'balanceDict:', balanceDict)
    np.set_printoptions(suppress=True)
    history['date'] = date
    history['statusList'] = statusList
    history['{}_available'.format(A_symbol)] = A_available
    history['{}_position'.format(A_symbol)] = A_position
    history['{}_price'.format(A_symbol)] = A_price
    history['{}_available'.format(B_symbol)] = B_available
    history['{}_position'.format(B_symbol)] = B_position
    history['{}_price'.format(B_symbol)] = B_price
    history['init'] = [200000]* history.shape[0]
    history['balance'] = history['{}_available'.format(A_symbol)] + (abs(history['{}_position'.format(A_symbol)]) * history['{}_price'.format(A_symbol)]) + history['{}_available'.format(B_symbol)] + (abs(history['{}_position'.format(B_symbol)]) * history['{}_price'.format(B_symbol)])
    history['pnl'] = history['balance'] - history['init']
    history['return'] = history['balance']/200000 -1
    history.to_csv("pair_trade_ratio/{}-{}_ratio.xls".format(A_symbol, B_symbol))
    # print(history['return'])
    print('[pair trade]{}-{} {}%'.format(A_symbol, B_symbol, "{:.8f}".format(history['return'].iloc[-1] * 100)))
    plt.clf()
    history['return'].plot()
    #     plt.show()
    plt.savefig("pair_trade_ratio/{}-{}_ratio".format(A_symbol, B_symbol))